In [ ]:
!pip install pyngrok

In [ ]:
!ngrok authtoken 2b7Ilnk9Ki0v9Rm63sKaZzJrNzl_6rKfcUhpchy3LbtwZqhyH

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from pyngrok import ngrok
ngrok.kill()

In [ ]:
from flask import Flask, request, jsonify
from flask import render_template, send_from_directory
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
import joblib
from threading import Thread

In [ ]:

df = pd.read_csv('/content/drive/MyDrive/animes.csv')

In [ ]:
df = df.drop_duplicates(['uid', 'title'], keep='first')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
tfidf_vectorizer = joblib.load('/content/drive/MyDrive/Colab Notebooks/tfidf_vectorizer.pkl')
tfidf_matrix = joblib.load('/content/drive/MyDrive/Colab Notebooks/tfidf_matrix.pkl')
knn_model = joblib.load('/content/drive/MyDrive/Colab Notebooks/knn_model.pkl')
knn_model.fit(tfidf_matrix)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=10)

In [ ]:
def get_hybrid_recommendations(user_id, anime_title, top_n=10):
    anime_index = df[df['title'] == anime_title].index
    if len(anime_index) == 0:
        return [], []

    collaborative_filtering_indices = get_collaborative_filtering_recommendations(anime_index)

    collaborative_filtering_indices = collaborative_filtering_indices[1:]

    collaborative_filtering_recommendations = df.iloc[collaborative_filtering_indices]['title'].tolist()

    return collaborative_filtering_recommendations[:top_n]

def get_collaborative_filtering_recommendations(anime_index):
    distances, indices = knn_model.kneighbors(tfidf_matrix[anime_index], n_neighbors=10)
    return indices.flatten()

In [ ]:
user_id = 5530
anime_title = 'Pandora Hearts'
collaborative_filtering_recs = get_hybrid_recommendations(user_id, anime_title)
print("Collaborative Filtering Recommendations:", collaborative_filtering_recs)

Collaborative Filtering Recommendations: ['Oz no Mahoutsukai (1986)', 'OZ', 'Oz no Mahoutsukai', 'Made in Abyss', 'Oz no Mahoutsukai no Koutsuu Anzen no Tabi', 'Space Oz no Bouken', 'Code Geass: Soubou no Oz Picture Drama', 'Summer Wars', 'Zhandou Wang Zhi Jufeng Zhan Hun']


In [ ]:
def convert_to_python_types(dataframe):
    for column in dataframe.select_dtypes(include=['int64']).columns:
        dataframe[column] = dataframe[column].astype(int)
    return dataframe


In [ ]:
from pyngrok import ngrok
import threading

ngrok.kill()
def start_ngrok():
    ngrok_address = ngrok.connect(5000)
    print(' * Tunnel URL:', ngrok_address)

threading.Thread(target=start_ngrok).start()

app = Flask(__name__)
def preprocess_input(anime_title):
    return anime_title
@app.route('/')
def index():
    return render_template('index.html', anime_data=df.to_dict(orient='records'))

@app.route('/anime_details')
def anime_details():
    return render_template('anime_details.html')

@app.route('/get_anime_data', methods=['POST'])
def get_anime_data():
    uid = int(request.form.get('user_id'))
    title = request.form.get('title')
    titles = get_hybrid_recommendations(uid, title)
    print(titles)

    anime_details = df[df['uid'] == int(uid)].iloc[0]

    details = {
        'uid': int(anime_details['uid']),
        'title': str(anime_details['title']),
        'synopsis': str(anime_details['synopsis']),
        'genre': str(anime_details['genre']),
        'aired': str(anime_details['aired']),
        'img_url': str(anime_details['img_url']),
        'link': str(anime_details['link']),
    }

    recommendations = df[df['title'].isin(titles)].copy()
    recommendations = convert_to_python_types(recommendations)

    recommendations_data = []
    for _, row in recommendations.iterrows():
        recommendations_data.append({
            'uid': int(row['uid']),
            'title': str(row['title']),
            'img_url': str(row['img_url']),
            'link': str(row['link']),
        })

    return jsonify({'anime_details': details, 'recommendations': recommendations_data})

@app.route('/get_anime_details', methods=['POST'])
def get_anime_details():
    uid = request.form.get('uid')
    title = request.form.get('title')
    titles = get_hybrid_recommendations(uid, title)
    anime_details = df[df['uid'] == int(uid)].iloc[0]

    title = anime_details['title']
    synopsis = anime_details['synopsis']
    genre = anime_details['genre']
    aired = anime_details['aired']
    episodes = anime_details['episodes']
    members = anime_details['members']
    popularity = anime_details['popularity']
    ranked = anime_details['ranked']
    score = anime_details['score']
    img_url = anime_details['img_url']
    link = anime_details['link']

    return render_template('anime_details.html',
                           uid=uid,
                           title=title,
                           synopsis=synopsis,
                           genre=genre,
                           aired=aired,
                           episodes=episodes,
                           members=members,
                           popularity=popularity,
                           ranked=ranked,
                           score=score,
                           img_url=img_url,
                           link=link)


@app.route('/get_recommendations', methods=['POST'])
def get_recommendations():
    try:
        user_id = int(request.form['user_id'])
        anime_title = request.form['anime_title']

        anime_title = preprocess_input(anime_title)

        anime_index = df[df['title'] == anime_title].index
        print(f"Anime Title: {anime_title}, Anime Index: {anime_index}")

        if len(anime_index) == 0:
            return jsonify({'error': 'Anime not found'}), 404

        collaborative_filtering_recs = get_hybrid_recommendations(user_id, anime_title)
        return jsonify({'user_id': user_id, 'anime_title': anime_title, 'recommendations': collaborative_filtering_recs})


    except Exception as e:
        return jsonify({'error': str(e)})

if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Tunnel URL: NgrokTunnel: "https://3d64-35-245-45-41.ngrok-free.app" -> "http://localhost:5000"


INFO:werkzeug:127.0.0.1 - - [24/Jan/2024 15:18:16] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Jan/2024 15:18:18] "GET /static/css/index.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Jan/2024 15:18:18] "GET /styles.css HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [24/Jan/2024 15:18:23] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [24/Jan/2024 15:20:04] "GET /anime_details?uid=40746&title=Overflow HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Jan/2024 15:20:06] "POST /get_anime_data HTTP/1.1" 200 -


['Natsu-iro Egao de 1, 2, Jump!', 'Million Doll', 'Love Live! School Idol Project OVA', 'Wonderful Rush', 'Music S.T.A.R.T!!', 'Netorare Zuma', 'Idol Memories', 'Koi-ken!: Watashitachi Anime ni Nacchatta!', 'Zou no Senaka: Tabidatsu Hi']


INFO:werkzeug:127.0.0.1 - - [24/Jan/2024 15:20:08] "GET /undefined HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [24/Jan/2024 15:22:40] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Jan/2024 15:25:45] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Jan/2024 15:25:46] "GET /styles.css HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [24/Jan/2024 15:25:46] "GET /static/css/index.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Jan/2024 15:25:58] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [24/Jan/2024 15:26:58] "GET /anime_details?uid=40746&title=Overflow HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Jan/2024 15:26:58] "POST /get_anime_data HTTP/1.1" 200 -


['Natsu-iro Egao de 1, 2, Jump!', 'Million Doll', 'Love Live! School Idol Project OVA', 'Wonderful Rush', 'Music S.T.A.R.T!!', 'Netorare Zuma', 'Idol Memories', 'Koi-ken!: Watashitachi Anime ni Nacchatta!', 'Zou no Senaka: Tabidatsu Hi']


INFO:werkzeug:127.0.0.1 - - [24/Jan/2024 15:26:59] "GET /undefined HTTP/1.1" 404 -


In [ ]:
print(tfidf_matrix.shape)


(16216, 43913)
